In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### Action operations on pair RDDs ???

- **rdd1.subtractByKey(rdd2)**: Return an RDD with the pairs whose keys are not in otherDataset.
- **rdd1.join(rdd2)**
- **rightOuterJoin(rdd2)**: Perform a join between two RDDs where the key must be present in the first RDD.
- **leftOuterJoin(rdd2)**: Perform a join between two RDDs where the key must be present in the other RDD
- **cogroup(rdd2)**: Group data from both RDDs sharing the same key

#### Exercise 1

Using “filtered_registered_business_sf.csv” and “supervisor_sf.csv”, list business names without a district supervisor.

In [4]:
text1 = sc.textFile('/Users/owner/USF/spark/filtered_registered_business_sf.csv')
text1.take(5)

[u'94123,Tournahu George L,3301 Broderick St,San Francisco,CA',
 u'94124,Stephens Institute Inc,2225 Jerrold Ave,San Francisco,CA',
 u'94105,Stephens Institute Inc,180 New Montgomery St,San Francisco,CA',
 u'94108,Stephens Institute Inc,540 Powell St,San Francisco,CA',
 u'94107,Stephens Institute Inc,460 Townsend St,San Francisco,CA']

In [3]:
text2 = sc.textFile('/Users/owner/USF/spark/supervisor_sf.csv')
text2.take(5)

[u'94102,8', u'94102,6', u'94102,3', u'94102,5', u'94103,8']

In [7]:
zip_business_name_pair = text1.map(lambda x: (x.split(",")[0],x.split(",")[1])).distinct().sortByKey(ascending = False)
zip_business_name_pair.take(10)

[(u'99999', u'Bayview Entertainment Society'),
 (u'99999', u'Rainforest Action Network'),
 (u'99999', u'Hunters Point Community Youth'),
 (u'99999', u'Sf Global Trade Council'),
 (u'99999', u'Society Of Neurological Surg'),
 (u'99999', u'U First Community Services'),
 (u'99999', u'Arlington Residences Inc'),
 (u'99999', u'Think 12 Corporation'),
 (u'99999', u'Admasu Menlikalew'),
 (u'99999', u'Epiphany Prodctns Sonic Dance')]

In [10]:
zip_sv = text2.map(lambda x: (x.split(",")[0], x.split(",")[1])).distinct().sortB
zip_sv.take(5)

[(u'94158', u'10'),
 (u'94158', u'6'),
 (u'94134', u'9'),
 (u'94134', u'11'),
 (u'94134', u'10')]

In [23]:
zip_no_sv = zip_business_name_pair.subtractByKey(zip_sv).values().distinct()

In [24]:
zip_no_sv.take(10)

[u'Precision Communication Serv',
 u'Schefer Thomas R',
 u'Jacob Abraham',
 u'Daniel Dela Rosa',
 u'Batista Luis S',
 u'"Wti',
 u'Boutin Jacqueline M',
 u'Avinesh P Singh',
 u'Edhuar Arellano',
 u'Ryann Kass']

#### Exercise 2: 

first_num_pairs = sc.parallelize({(2,3),(1,2),(1,3),(2,4),(3,6)})

second_num_pairs = sc.parallelize({(1,3),(2,2)})

What are the results of ...

1. first_num_pairs.subtract(second_num_pairs)
2. first_num_pairs.subtractByKey(second_num_pairs)
3. first_num_pairs.join(second_num_pairs)
4. first_num_pairs.rightOuterJoin(second_num_pairs)
5. first_num_pairs.leftOuterJoin(second_num_pairs)
6. first_num_pairs.cogroup(second_num_pairs)

In [25]:
# Initialize
first_num_pairs = sc.parallelize({(2,3),(1,2),(1,3),(2,4),(3,6)})
second_num_pairs = sc.parallelize({(1,3),(2,2)})

In [26]:
first_num_pairs.collect()

[(1, 2), (1, 3), (2, 3), (2, 4), (3, 6)]

In [27]:
second_num_pairs.collect()

[(1, 3), (2, 2)]

In [25]:
first_num_pairs.subtract(second_num_pairs).collect() # should get [(1,2), (2,3), (2,4), (3,6)]

[(1, 2), (2, 4), (2, 3), (3, 6)]

**subtractByKey()**:

Return an RDD with the pairs whose keys are not in otherDataset

In [28]:
first_num_pairs.subtractByKey(second_num_pairs).collect() # should get: [(3,6)]

[(3, 6)]

In [29]:
first_num_pairs.join(second_num_pairs).collect() #[(1,(2,3)), (1, (3,3)),(2,(2,3)), (2,(3,2)), (2,(4,2))]

[(1, (2, 3)), (1, (3, 3)), (2, (3, 2)), (2, (4, 2))]

In [28]:
first_num_pairs.rightOuterJoin(second_num_pairs).collect() 

[(1, (2, 3)), (1, (3, 3)), (2, (3, 2)), (2, (4, 2))]

In [29]:
first_num_pairs.leftOuterJoin(second_num_pairs).collect()

[(1, (2, 3)), (1, (3, 3)), (2, (3, 2)), (2, (4, 2)), (3, (6, None))]

**cogroup(otherDataset)**:
 - group data form two data sets sharing the same key
 - go over two RDDs sharing the same key
 - return list of pairs: (Key, (ResultIterable, ResultIterable))

In [30]:
first_num_pairs.cogroup(second_num_pairs).collect()

[(1,
  (<pyspark.resultiterable.ResultIterable at 0x10e6870d0>,
   <pyspark.resultiterable.ResultIterable at 0x10e67fa50>)),
 (2,
  (<pyspark.resultiterable.ResultIterable at 0x10e67f610>,
   <pyspark.resultiterable.ResultIterable at 0x10e67fb50>)),
 (3,
  (<pyspark.resultiterable.ResultIterable at 0x10e67f590>,
   <pyspark.resultiterable.ResultIterable at 0x10e67fbd0>))]

Now, to actually present the results of the __cogroup()__ operation, apply a __map__ function and don't forget to turn each of the < RI > into a list!

In [30]:
first_num_pairs.cogroup(second_num_pairs).map(lambda x: (x[0],(list(x[1][0]),list(x[1][1])))).collect()

[(1, ([2, 3], [3])), (2, ([3, 4], [2])), (3, ([6], []))]

In [33]:
first_num_pairs.countByKey()

defaultdict(int, {1: 2, 2: 2, 3: 1})

#### Example 3

Using Example 1, list (zip,(business_name, supervisor_id)) pairs ordered by supervisor_id.

- Only if both business and supervisor exist.
- If a business exists.
- If a supervisor exists.

In [32]:
#3.1. Only if both business and supervisor exist.
both_exist = zip_business_name_pair.join(zip_sv)
both_exist.sortBy(lambda x: x[1][1], ascending = False).take(10)


[(u'94112', (u'Tradicion Peruana Cultural Cnt', u'9')),
 (u'94112', (u'Houston Victorene P', u'9')),
 (u'94112', (u'Sole Prop Daniel Castellanos', u'9')),
 (u'94112', (u'Baysac Belanio G & Carmen S', u'9')),
 (u'94112', (u'Piskov Oksana', u'9')),
 (u'94112', (u'Navarro Miguel & Joanne', u'9')),
 (u'94112', (u'Jing Luo', u'9')),
 (u'94112', (u'Bustillo Gloria Maria Ulloa', u'9')),
 (u'94112', (u'Lai Seu Wannie/gu Wei', u'9')),
 (u'94112', (u'Chiu Chong Ying', u'9'))]

In [39]:
#3.2. If a business exists
business_exists = zip_business_name_pair.leftOuterJoin(zip_sv)
len(business_exists.sortBy(lambda x: x[1][1], ascending=False).collect())

439955

In [38]:
#3.3. If a supervisor exists
sv_exists = zip_business_name_pair.rightOuterJoin(zip_sv)
len(sv_exists.sortBy(lambda x: x[1][1], ascending=False).collect())

398813

#### Example 4
Generate Key(zip) and value pair RDDs from “filtered_registered_business_sf.csv” and “supervisor_sf.csv” and cogroup() the RDDs.

In [46]:
cogroup_rdds = zip_business_name_pair.cogroup(zip_sv)
cogroup_rdds.map(lambda x: (x[0],(list(x[1][0]),list(x[1][1])))).take(2)

[(u'',
  ([u'Opower Inc',
    u'Poco Petroleum Inc',
    u'Rice Paradise Llc',
    u'Cantrell Harris & Assoc Inc',
    u'Cooper Jim B',
    u'Arconas Corporation',
    u'Woo H Woo S',
    u'221 7th Street Residences Llc',
    u'1625 Leavenworth Street Llc',
    u'Bond Blacktop Inc',
    u'Bibliocommons Inc',
    u'Ortiz Jose E',
    u'Eldred Robert J Cfp',
    u'Endurance Wind Power Inc',
    u'Joseph D & Candice M Harney Trust Jc & Cm Harney Trust',
    u'Cardno Entrix',
    u'Paybyphone Technologies Inc',
    u'Willis Supply Corporation',
    u'New Flyer Industriescanada Ulc',
    u'Vip Plumbing And Drain Cleanin',
    u'Htut Chris',
    u'Mid Canada Millwork Ltd',
    u'Act Fuels Inc',
    u'Allstream Inc',
    u'Radio Ip Software Inc',
    u'C Fischer And Sons Llc',
    u'Abd Insurance & Financial Serv',
    u'Built 1925 Llc',
    u'Ramon Chavez',
    u'Cirque Du Soleil Inc',
    u'Law Office Of Scott A Sommer',
    u'Moonka Nishi',
    u'Hampton Court Sf Lp',
    u'Margaret Apartm

In [48]:
cogroup_rdds.map(lambda x: (x[0],(len(list(x[1][0])),len(list(x[1][1]))))).collect()

[(u'', (68, 0)),
 (u'85716', (1, 0)),
 (u'19443', (1, 0)),
 (u'28036', (1, 0)),
 (u'55344', (5, 0)),
 (u'92532', (1, 0)),
 (u'44122', (8, 0)),
 (u'95627', (4, 0)),
 (u'94008', (1, 0)),
 (u'95623', (1, 0)),
 (u'53562', (5, 0)),
 (u'93953', (2, 0)),
 (u'10516', (1, 0)),
 (u'91748', (3, 0)),
 (u'94615', (1, 0)),
 (u'20854', (1, 0)),
 (u'94611', (265, 0)),
 (u'77010', (2, 0)),
 (u'32202', (3, 0)),
 (u'94619', (189, 0)),
 (u'92653', (6, 0)),
 (u'73018', (1, 0)),
 (u'92657', (1, 0)),
 (u'40517', (1, 0)),
 (u'22180', (1, 0)),
 (u'94581', (1, 0)),
 (u'56716', (1, 0)),
 (u'91304', (4, 0)),
 (u'19403', (2, 0)),
 (u'90242', (1, 0)),
 (u'32714', (1, 0)),
 (u'96150', (6, 0)),
 (u'94578', (254, 0)),
 (u'91344', (1, 0)),
 (u'46818', (2, 0)),
 (u'95678', (41, 0)),
 (u'93603', (1, 0)),
 (u'94574', (14, 0)),
 (u'94808', (2, 0)),
 (u'46825', (1, 0)),
 (u'48197', (1, 0)),
 (u'94800', (1, 0)),
 (u'85253', (1, 0)),
 (u'94804', (457, 0)),
 (u'96019', (1, 0)),
 (u'2895', (1, 0)),
 (u'55379', (1, 0)),
 (u'9353

#### Example 5:

From ”filtered_registered_business_sf.csv”, create a pair RDD of (zip, (store name, city))
- Count pairs which do not have a key.
- Filter pairs that do not include “San Francisco” in the city value.

In [49]:
text1.take(10)

[u'94123,Tournahu George L,3301 Broderick St,San Francisco,CA',
 u'94124,Stephens Institute Inc,2225 Jerrold Ave,San Francisco,CA',
 u'94105,Stephens Institute Inc,180 New Montgomery St,San Francisco,CA',
 u'94108,Stephens Institute Inc,540 Powell St,San Francisco,CA',
 u'94107,Stephens Institute Inc,460 Townsend St,San Francisco,CA',
 u'94109,Stephens Institute Inc,1835-49 Van Ness Ave,San Francisco,CA',
 u'94102,Stephens Institute Inc,620 Sutter St,San Francisco,CA',
 u'94102,Stephens Institute Inc,655 Sutter St,San Francisco,CA',
 u'94109,Stephens Institute Inc,1055 Pine St,San Francisco,CA',
 u'94107,Stephens Institute Inc,121 Wisconsin St,San Francisco,CA']

In [57]:
zip_store_city = text1.map(lambda x: x.split(",")).map(lambda x: (x[0], (x[1],x[3])))
zip_store_city.take(5)

[(u'94123', (u'Tournahu George L', u'San Francisco')),
 (u'94124', (u'Stephens Institute Inc', u'San Francisco')),
 (u'94105', (u'Stephens Institute Inc', u'San Francisco')),
 (u'94108', (u'Stephens Institute Inc', u'San Francisco')),
 (u'94107', (u'Stephens Institute Inc', u'San Francisco'))]

In [63]:
zip_store_city.countByKey() # there are 92 pairs without a zip

defaultdict(int,
            {u'': 92,
             u'95127': 53,
             u'77046': 1,
             u'55344': 5,
             u'55346': 1,
             u'55343': 11,
             u'93257': 1,
             u'92102': 6,
             u'60064': 2,
             u'92653': 6,
             u'92651': 4,
             u'92657': 1,
             u'92656': 6,
             u'92879': 5,
             u'19406': 3,
             u'32837': 3,
             u'19403': 2,
             u'19711': 1,
             u'92870': 4,
             u'94203': 1,
             u'95121': 63,
             u'32301': 1,
             u'32309': 1,
             u'32308': 1,
             u'74066': 1,
             u'53029': 1,
             u'93430': 1,
             u'10965': 1,
             u'96155': 1,
             u'93436': 1,
             u'10960': 1,
             u'46131': 1,
             u'65652': 1,
             u'98642': 1,
             u'99999': 10,
             u'90249': 5,
             u'60131': 1,
             u'92821'

In [65]:
#pairs which do not have a key
zip_store_city.lookup("")[:10]

[(u'Hartmann Studios Incorporated', u''),
 (u'Cardno Entrix', u''),
 (u'Bond Blacktop Inc', u''),
 (u'Moonka Nishi', u''),
 (u'Cooper Jim B', u''),
 (u'Ascencion Flores Ismael O', u''),
 (u'Opower Inc', u''),
 (u'Htut Chris', u''),
 (u'Red Oxygen Inc', u''),
 (u'Miniclip America Inc', u'"London')]

In [66]:
#Filter pairs that do not include “San Francisco” in the city value.
zip_store_city.filter(lambda x: "San Francisco" not in x[1][1]).collect()[:10]


[(u'94108', (u'"Fugazi Travel Agency', u'170 Grant Ave 4th Fl')),
 (u'94111', (u'"Fugazi Travel Agency', u'400 Sansome St')),
 (u'94111', (u'"Fugazi Travel Agency', u'400 Sansome St')),
 (u'94111', (u'"Fugazi Travel Agency', u'400 Sansome St')),
 (u'94111', (u'"Fugazi Travel Agency', u'400 Sansome St')),
 (u'94133', (u'Alioto Fish Co Ltd', u'San+francisco')),
 (u'94545', (u'Allan Auto Sprinkler Corp', u'Hayward')),
 (u'94062', (u'Antonchuk Richard A', u'Redwood+city')),
 (u'94132', (u'Atlantic Richfield Co', u'San+francisco')),
 (u'94014', (u'Aunt Anns Corp Headquarters', u'Daly+city'))]

## Tuning Spark  - Persist in Memory/Disk

RDDs are by default recomputed each time.

However, if you want to reuse an RDD for multiple actions, you can ask Spark to store the content in memory/disk and query repeatedly.

```line_with_spark.persist(StorageLevel.persistency_level)```

One of the most important capabilities in Spark is **persisting (or caching)** a dataset in memory across operations. When you persist an RDD, each node stores any partitions of it that it computes in memory and reuses them in other actions on that dataset (or datasets derived from it). This allows future actions to be much faster (often by more than 10x). Caching is a key tool for iterative algorithms and fast interactive use.

You can mark an RDD to be persisted using the ```persist()``` or ```cache()``` methods on it. The first time it is computed in an action, it will be kept in memory on the nodes. Spark’s cache is fault-tolerant.

In addition, each persisted RDD can be stored using a different storage level, allowing you, for example, to persist the dataset on disk, persist it in memory but as serialized Java objects (to save space), replicate it across nodes. These levels are set by passing a _StorageLevel object_ (Scala, Java, Python) to persist(). The cache() method is a shorthand for using the default storage level, which is StorageLevel.MEMORY_ONLY (store deserialized objects in memory). The full set of storage levels is:

**Note:** *In Python, stored objects will always be serialized with the Pickle library, so it does not matter whether you choose a serialized level. The available storage levels in Python include MEMORY_ONLY, MEMORY_ONLY_2, MEMORY_AND_DISK, MEMORY_AND_DISK_2, DISK_ONLY, and DISK_ONLY_2.*


|Storage Level|Meaning|
|--------------|------------|
|MEMORY_ONLY|Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, some partitions will not be cached and will be recomputed on the fly each time they're needed. This is the default level.|
|MEMORY_AND_DISK	|Store RDD as deserialized Java objects in the JVM. If the RDD does not fit in memory, store the partitions that don't fit on disk, and read them from there when they're needed.
|MEMORY_ONLY_SER|(Java and Scala)	Store RDD as serialized Java objects (one byte array per partition). This is generally more space-efficient than deserialized objects, especially when using a fast serializer, but more CPU-intensive to read.
|MEMORY_AND_DISK_SER|(Java and Scala)	Similar to MEMORY_ONLY_SER, but spill partitions that don't fit in memory to disk instead of recomputing them on the fly each time they're needed.
|DISK_ONLY|	Store the RDD partitions only on disk.
|MEMORY_ONLY_2, MEMORY_AND_DISK_2, etc.|Same as the levels above, but replicate each partition on two cluster nodes.
|OFF_HEAP (experimental)|Similar to MEMORY_ONLY_SER, but store the data in off-heap memory. This requires off-heap memory to be enabled.

You can use **```.persist(storage_level)```**

OR

**```.cache() = .persist(MEMORY_ONLY) = .Persist()```**

To change form MEMORY_ONLY to MEMORY_AND_DISK, you have to first run **```.unpersist()```**,  otherwise Error will be thrown up.

To check the persistence level

- **getStorageLevel()**: checks if it uses Disk, Memory , at external system, as deserialized Java objects, replication
    - _StorageLevel(useDisk, useMemory, useOffHeap, deserialized, replication = 1)_
        - useDisk : If set, partitions that do not fit in memory will be written to disk.==> use only if recomputation is expensive and cannot fit the memory 
        - useMemory : If set, the RDDs will be stored in-memory.
        - useOffHeap : If set, the RDD will be stored outside of the Spark executor in an external system such as Tachyon ==> use when there is memory issue/noisy cluster (when somehting happens to your cluster)
        - deserialization : If set, the RDD will be stored as deserialized Java objects.
        - replication : An integer that controls the number of copies of the persisted data to be stored.

NOTES:

- **MEMORY_ONLY**  is the best

- **Disk**: 

- **off-Heap**: Stored outside of the Spark executor in an external system ==> use when there is memory issue/noisy cluster (when somehting happens to your cluster and 

- **Serialization** - it makes objects smaller. So, when the job is too big to fit into memory, serialization could be done

- **Replication** - faster fault recovery. Cost more space/speed ==> use when bad connection between the nodes happens often / live web app.
}


**cache()** is the same as calling **persist(MEMORY_ONLY)**.
- Once persisted, RDDs can be reused multiple times without requiring reevaluation (recalculation).
- If there is not enough memory available to cache the RDD, it will be reevaluated for each lineage triggered by an action.

**unpersist()** lets you manually remove them from the cache.

**persist()**

 - Many options (See Table) - memory/disk and replication
 -  When using disk options, the persisted data on disk can be used to reconstitute partitions lost due to executor or memory failure.
 
**unpersist()** lets you unpersist the RDD.

In [67]:
import os, time
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel 
sc = SparkContext.getOrCreate()

#### Example 6

Load a big file and perform an action.

In [68]:
file_name = os.path.expanduser("~/data/Train.csv")

In [69]:
lines = sc.textFile(file_name)

In [70]:
lines.getStorageLevel()

StorageLevel(False, False, False, False, 1)

In [71]:
#without persisting
start = time.time()
lines.count()
print "first ",
print time.time() - start

start = time.time()
lines.count()
print "second ",
print time.time() - start

first  2.00486493111
second  1.30234479904


In [72]:
#with memory
lines.cache()
start = time.time()
lines.count()
print "first ",
print time.time() - start #trigger reevaluation

start = time.time()
lines.count()
print "second ",
print time.time() - start #doesn't trigger reevaluation

first  2.10643815994
second  1.2384428978


In [73]:
lines.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [74]:
lines.unpersist()

/Users/owner/data/Train.csv MapPartitionsRDD[327] at textFile at NativeMethodAccessorImpl.java:0

In [75]:
#with memory and disk
lines.persist(StorageLevel.MEMORY_AND_DISK)
start = time.time()
lines.count()
print "first ",
print time.time() - start #trigger reevaluation

start = time.time()
lines.count()
print "second ",
print time.time() - start #doesn't trigger reevaluation

start = time.time()
lines.count()
print "third ",
print time.time() - start #doesn't trigger reevaluation

start = time.time()
lines.count()
print "fourth ",
print time.time() - start #doesn't trigger reevaluation

first  2.52633690834
second  1.2457408905
third  1.12653112411
fourth  1.22146892548


In [76]:
lines.getStorageLevel()

StorageLevel(True, True, False, False, 1)

In [77]:
lines.unpersist()

/Users/owner/data/Train.csv MapPartitionsRDD[327] at textFile at NativeMethodAccessorImpl.java:0

In [78]:
lines.cache()
lines.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [81]:
lines.unpersist()
lines.persist(StorageLevel(False,True, False, False, 3))
lines.getStorageLevel()

StorageLevel(False, True, False, False, 3)

#### Exercise 7:

Try different persistency levels on RDDs which include “Spark”.

In [83]:
lines = sc.textFile("README.md")

In [84]:
lines_spark = lines.filter(lambda line: "Spark" in line)

In [86]:
lines_spark.take(3)

[u'# Apache Spark',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'rich set of higher-level tools including Spark SQL for SQL and DataFrames,']

In [91]:
lines_spark.persist(StorageLevel.DISK_ONLY)
lines_spark.getStorageLevel()


StorageLevel(True, False, False, False, 1)

In [95]:
lines_spark.unpersist()

PythonRDD[342] at RDD at PythonRDD.scala:48

In [97]:
lines_spark.persist(StorageLevel.MEMORY_ONLY_2)
lines_spark.getStorageLevel()

StorageLevel(False, True, False, False, 2)

In [102]:
lines_spark.unpersist()

PythonRDD[342] at RDD at PythonRDD.scala:48

In [105]:
lines_spark.persist(StorageLevel.MEMORY_AND_DISK_SER)
lines_spark.getStorageLevel()

StorageLevel(True, True, False, False, 1)

### NOTES:
- If your RDDs fit comfortably with MEMORY_ONLY, leave them that way. This is the most CPU-efficient option, allowing operations on the RDDs to run as fast as possible. Iterative algorithms are often good candidates for caching.
- Don’t spill to disk unless the functions that computed your datasets are expensive, or they use a large amount of the data. Otherwise, recomputing a partition may be as fast as reading it from disk.
- Use the replicated storage levels if you want fast fault recovery (e.g. if using Spark to serve requests from a web application). All the storage levels provide full fault tolerance by recomputing lost data, but the replicated ones let you continue running tasks on the RDD without waiting to recompute a lost partition.

### Tips - Passing functions to Spark

Most of Spark’s transformations and actions depend on passing in functions that are used by Spark to compute data.
1. Pass in lambda functions : for shorter functions.
2. Pass in top-level functions or locally defined functions.